In [39]:
import requests
from datetime import datetime

In [2]:
API_KEY = 'ed663eae-ace1-4a03-b59a-2c4a6dde68a7'

In [12]:
url = 'https://url.santiagolarrain.myds.me'

In [13]:

session = requests.Session()

# Replace 'API_ENDPOINT' and 'API_KEY' with your actual API endpoint URL and API key
API_ENDPOINT = f'{url}/rest/v3/short-url'

def call_post_api_endpoint():
    headers = {
        'X-Api-Key': f'{API_KEY}',
        'Content-Type': 'application/json',
    }

    data = {
        # Replace with actual data you want to send in the POST request
        'longUrl': 'https://www.pacificlabs.cl/',
        'tags': ['pixel', 'testing']
    }

    response = session.post(API_ENDPOINT, headers=headers, json=data)

    if response.status_code == 200:
        print('API call successful!')
        print(response.json())
    else:
        print('Error:', response.status_code)
        print(response.text)

In [11]:
call_post_api_endpoint()

Error: 404
{"title":"Not Found","type":"https://shlink.io/api/error/not-found","status":404,"detail":"Cannot POST http://url.santiagolarrain.myds.me/v3/short-url!"}


In [31]:
url = "https://url.santiagolarrain.myds.me/rest/v3/short-urls"

payload = {
    "longUrl": "https://www.pacificlabs.cl",
    "tags": ["pixel", "testing"]
}
headers = {
    "accept": "application/json",
    "X-Api-Key": "ed663eae-ace1-4a03-b59a-2c4a6dde68a7",
    "Content-Type": "application/json"
}

response = requests.request("POST", url, json=payload, headers=headers)

print(response.text)

{"shortCode":"IKcPS","shortUrl":"https://url.santiagolarrain.myds.me/IKcPS","longUrl":"https://www.pacificlabs.cl","dateCreated":"2024-08-08T20:35:14-04:00","tags":["pixel","testing"],"meta":{"validSince":null,"validUntil":null,"maxVisits":null},"domain":null,"title":"PacificLabs","crawlable":false,"forwardQuery":true,"visitsSummary":{"total":0,"nonBots":0,"bots":0}}


In [32]:
response.json()

{'shortCode': 'IKcPS',
 'shortUrl': 'https://url.santiagolarrain.myds.me/IKcPS',
 'longUrl': 'https://www.pacificlabs.cl',
 'dateCreated': '2024-08-08T20:35:14-04:00',
 'tags': ['pixel', 'testing'],
 'meta': {'validSince': None, 'validUntil': None, 'maxVisits': None},
 'domain': None,
 'title': 'PacificLabs',
 'crawlable': False,
 'forwardQuery': True,
 'visitsSummary': {'total': 0, 'nonBots': 0, 'bots': 0}}

In [48]:
print (datetime.strptime(response.json()['dateCreated'], '%Y-%m-%dT%H:%M:%S%z'))

2024-08-08 20:35:14-04:00


In [53]:
from mysql import connector

import requests
import configparser


def make_pixel_and_save_to_mariadb():

    config = configparser.ConfigParser()
    config.read('./credentials.ini', )

    user = config['NASDB']['user']
    password=config['NASDB']['password']
    host=config['NASDB']['host']
    port=config['NASDB']['port'] 
    database=config['NASDB']['database']

    url = "https://url.santiagolarrain.myds.me/rest/v3/short-urls"
    
    payload = {
        "longUrl": "https://www.pacificlabs.cl",
        "tags": ["pixel", "testing"]
    }
    headers = {
        "accept": "application/json",
        "X-Api-Key": API_KEY,
        "Content-Type": "application/json"
    }
    
    r = requests.request("POST", url, json=payload, headers=headers)

    print (r)
    data = r.json()
    print (data)
    
    url = data['shortUrl']
    date_created = datetime.strptime(data['dateCreated'], '%Y-%m-%dT%H:%M:%S%z')
    short_code = data['shortCode']

    conn = connector.connect(
        user=user,
        password=password,
        host=host,
        port=port,
        database=database
        )
    cur = conn.cursor() 

    # #retrieving information 
    # some_name = "Georgi" 
    # cur.execute("SELECT first_name,last_name FROM employees WHERE first_name=?", (some_name,)) 

    # for first_name, last_name in cur: 
        # print(f"First name: {first_name}, Last name: {last_name}")
        
    stmt = "INSERT INTO pixel_email (url, shortcode, created_date) VALUES (%s, %s, %s)"
    #insert information 
    try: 
        cur.execute(stmt, [url, short_code, date_created])
    except Exception as e: 
        print(f"Error: {e}")

    conn.commit() 
    # print(f"Last Inserted ID: {cur.lastrowid}")
        
    conn.close()

In [54]:
make_pixel_and_save_to_mariadb()

<Response [200]>
{'shortCode': 'TPbAu', 'shortUrl': 'https://url.santiagolarrain.myds.me/TPbAu', 'longUrl': 'https://www.pacificlabs.cl', 'dateCreated': '2024-08-09T13:31:14-04:00', 'tags': ['pixel', 'testing'], 'meta': {'validSince': None, 'validUntil': None, 'maxVisits': None}, 'domain': None, 'title': 'PacificLabs', 'crawlable': False, 'forwardQuery': True, 'visitsSummary': {'total': 0, 'nonBots': 0, 'bots': 0}}
